In [5]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 1, 2: 1, 3: 1, 4: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4}


In [12]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os 
import numpy as np

In [7]:
vocab_size = 88584
max_len = 250
batch_size = 64
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size)

In [8]:
train_data

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1

In [15]:
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data, max_len)
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, max_len)

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 86s 136ms/step - loss: 0.4134 - acc: 0.8086 - val_loss: 0.2986 - val_acc: 0.8842
Epoch 2/10
625/625 [==============================] - 81s 129ms/step - loss: 0.2360 - acc: 0.9097 - val_loss: 0.4773 - val_acc: 0.8446
Epoch 3/10
625/625 [==============================] - 75s 121ms/step - loss: 0.1811 - acc: 0.9336 - val_loss: 0.4953 - val_acc: 0.8094
Epoch 4/10
625/625 [==============================] - 75s 121ms/step - loss: 0.1531 - acc: 0.9457 - val_loss: 0.3849 - val_acc: 0.8564
Epoch 5/10
625/625 [==============================] - 76s 122ms/step - loss: 0.1337 - acc: 0.9531 - val_loss: 0.2948 - val_acc: 0.8904
Epoch 6/10
625/625 [==============================] - 75s 120ms/step - loss: 0.1120 - acc: 0.9619 - val_loss: 0.3120 - val_acc: 0.8862
Epoch 7/10
625/625 [==============================] - 72s 115ms/step - loss: 0.0970 - acc: 0.9671 - val_loss: 0.2971 - val_acc: 0.8852
Epoch 8/10
625/625 [==============================] - 7

In [23]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 40s 51ms/step - loss: 0.4116 - acc: 0.8489
[0.4115636646747589, 0.8488799929618835]


In [25]:
word_index = imdb.get_word_index()
def encode_text(text):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return tf.keras.preprocessing.sequence.pad_sequences([tokens], max_len)[0]

In [26]:
text = "that movie was so amazing, just amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  3

In [33]:
reverse_word_index = {value:key for (key, value) in word_index.items()}
def decode_text(integers):
    pad =0
    text = ""
    for num in integers:
        if num!=pad:
            text += reverse_word_index[num]+" "
            
    return text[:-1]
print(decode_text(encoded))

that movie was so amazing just amazing


In [35]:
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250))
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])
    
positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 229ms/step
[0.9694247]
1/1 [==============================] - 0s 24ms/step
[0.47167164]
